In [1]:
import numpy as np
import pandas as pd

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,TimestampType
from pyspark.sql.functions import col,current_timestamp,to_date,hour,dayofweek
spark = SparkSession \
    .builder \
    .master('local[*]') \
    .config("spark.driver.memory", "16g") \
    .appName('my-cool-app') \
    .getOrCreate()

sc=spark.sparkContext

23/01/22 17:17:37 WARN Utils: Your hostname, abdi-VivoBook-ASUSLaptop-X415EP-R465EP resolves to a loopback address: 127.0.1.1; using 172.20.40.193 instead (on interface wlo1)
23/01/22 17:17:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/22 17:17:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
schema = StructType([ \
        StructField("DEVICE_CODE", StringType(), True), 
        StructField("SYSTEM_ID",IntegerType(),True), \
        StructField("ORIGINE_CAR_KEY",StringType(),True), \
        StructField("FINAL_CAR_KEY",StringType(),True), \
        StructField("CHECK_STATUS_KEY", IntegerType(), True), \
        StructField("COMPANY_ID", StringType(), True), \
        StructField("PASS_DAY_TIME", TimestampType(), True)
    ])

In [4]:
df=spark.read.csv('TrafficData.csv',header=True,schema=schema)
df.show(1)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|   22010122|      284|       97955760|     64111706|               7|       161|2022-01-10 08:58:02|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
only showing top 1 row



In [5]:
Traffic = df.rdd.map(lambda x: ((x['FINAL_CAR_KEY'] , x['PASS_DAY_TIME'].date()), x['DEVICE_CODE']))


In [6]:
Traffic = Traffic.groupByKey()
del df

In [7]:
Traffic = Traffic.map(lambda x : (x[0],sorted(set(list(x[1])))))

In [9]:
from operator import add
#like before we are making list of camera for each car and we only keep list of cameras because we want to analys cameras
Traffic_temp = Traffic.map(lambda x : tuple(x[1]))


In [10]:
#I calculate this number from maximum length of cameras list for each car
# and we need this because we want to make a logical random road and this is useful for calculating probability
# it is a constant value so i calculate it once and because of saving resource Im just writing it
max_value = 825


In [12]:
#distinct_values of camera
distinct_val = Traffic_temp.flatMap(lambda x : x).distinct().collect()
len(distinct_val)

946

In [13]:
#assigning each camera_id an index
dict_word = {}
for i , str_temp in enumerate(distinct_val):
    dict_word[str_temp] = i

In [14]:

#making a logical random road
#for having a camera in some random road i calculate max_num/2 over all distinct cameras length
random_road = np.zeros(len(distinct_val))
count_road = 0
prob_num = max_value / len(distinct_val)
for i in range(len(random_road)):
    t = np.random.rand(1)
    if t[0] < prob_num / 2:
        random_road[i] = 1
        count_road += 1
random_road

array([0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1.,
       0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0.,
       1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1.,
       0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1.,
       1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 0.

In [15]:
#just defining cosine distance
def make_cosine_dist(x_in , random_road):
    in_list = x_in[1]
    sum_temp = 0
    for item in in_list:
        sum_temp += random_road[dict_word[item]]
    num = sum_temp / (np.sqrt(count_road * len(in_list)))
    return ((1 - (np.arccos(num)/np.pi) , x_in[0][0] , x_in[0][1] , x_in[1]))




In [16]:
#here we want to see which roads are most similar to our random road
#at first I printed 5 most similar roads that have unique car_id
#but then I've just printed 5 most similar roads
#and if we see result 5 most similar road is for just one car_id and it is probably his road to workplace or ...
Traffic_embed = Traffic.map(lambda x : make_cosine_dist(x, random_road)).sortBy(lambda x : -x[0])
most_similar = Traffic_embed.take(1000)
car_unique_list = []
for item in most_similar:
    if item[1] in car_unique_list:
        pass
    else:
        car_unique_list.append(item[1])
        print("for car {} in date : {} with device list of : {} similarity is : {}".format(item[1],item[2] , item[3] , item[0]))

for i in range(5):
    item = most_similar[i]
    print("for car {} in date : {} with device list of : {} similarity is : {}".format(item[1],item[2] , item[3] , item[0]))

for car 64111706 in date : 2022-01-10 with device list of : ['100', '1001002', '1001003', '1001004', '1001007', '1001008', '1001010', '1001017', '1001023', '1001024', '1001028', '1001038', '1001040', '1001042', '1001043', '1001044', '1001047', '1001051', '1001061', '1001063', '1001068', '1001073', '1001078', '1001079', '10012805', '10012806', '10015001', '10015002', '10015101', '10015201', '10015302', '10015401', '100700804', '100700812', '100700814', '100700818', '100700820', '100700826', '100700834', '100700835', '100700836', '100700838', '100700839', '100700841', '100700847', '100700853', '100700857', '100700864', '100700871', '100700880', '100700881', '100700894', '100700898', '100700899', '10070090', '100700905', '100700910', '100700911', '100700914', '100700918', '100700919', '100700920', '100700921', '100700924', '100700925', '100700927', '100700928', '100700929', '100700932', '100700934', '100700937', '100700938', '100700943', '100700944', '100700948', '100700950', '100700957',

In [17]:
del Traffic_embed

In [18]:
import gc
gc.collect()

18

In [19]:
r = 10
b = 20
hyperplanes = [[] for i in range(b*r)]
for j in range(b*r):
    hyperplanes[j] = [-1 for i in range(len(distinct_val))]
    for i in range(len(hyperplanes[j])):
        t = np.random.rand(1)
        if t[0] < 0.5:
            hyperplanes[j][i] = 1


In [20]:
gc.collect()

0

In [21]:
#using hyperplane_hash for cosine_distance function

def hyperplane_hash(x , r , b , hyperplanes):
    in_list = x[1]
    result = ['0' for i in range(b*r)]
    for j in range(b*r):
        hyperplane = hyperplanes[j]
        dot_prod = 0
        for item in in_list:
            if hyperplane[dict_word[item]] == 1:
                dot_prod += 1
            else:
                dot_prod -= 1
        
        if dot_prod < 0:
            result[j] = '0'
        else:
            result[j] = '1'
    out = []
    for p in range(r):
        start_ind = p*b
        end_ind = (p+1) * b 

        temp_str = result[start_ind:end_ind]
        num = 0
        for y in range(b):
            if temp_str[y] == '1':
                num += (2 ** y)
        out.append((p,num))
    return (x, out)


In [22]:
Traffic_embed = Traffic.map(lambda x : hyperplane_hash(x, 10, 20 , hyperplanes))


In [23]:
#making a hyperplane and accept this hyperplane to all results

random_result = []
for plane in hyperplanes:
    if np.dot(plane , random_road)>0:
        random_result.append('1')
    else:
        random_result.append('0')

hash_rand = []
for p in range(r):
    start_ind = p*b
    end_ind = (p+1) * b 

    temp_str = random_result[start_ind:end_ind]
    num = 0
    for y in range(b):
        if temp_str[y] == '1':
            num += (2 ** y)
    hash_rand.append((p,num))
hash_rand

[(0, 344697),
 (1, 914915),
 (2, 213927),
 (3, 390265),
 (4, 995236),
 (5, 191578),
 (6, 317881),
 (7, 153525),
 (8, 848525),
 (9, 1037432)]

In [24]:
#checking for candidates in hashing
def check_candidate(x , check_list):
    list_1 = x[1]
    for item in list_1:
        if item in check_list:
            return(x)
    return(0)




In [25]:
Traffic_embed = Traffic_embed.map(lambda x : check_candidate(x , hash_rand))
candidate_list = Traffic_embed.collect()


In [26]:
#if item was candidate then calues is not zero so we are printing all non_zero values
for item in candidate_list:
    if item != 0:
        print(item)
        

((('8826688', datetime.date(2022, 1, 9)), ['100700835', '231', '631367', '631634', '631763', '631795', '900104', '900155', '900225', '900243', '900255']), [(0, 876663), (1, 835042), (2, 886027), (3, 217709), (4, 995236), (5, 187497), (6, 303555), (7, 262290), (8, 669176), (9, 214176)])
((('10170784', datetime.date(2022, 1, 9)), ['100700826', '900104', '900236']), [(0, 862328), (1, 681651), (2, 381802), (3, 385658), (4, 995236), (5, 194131), (6, 324199), (7, 426738), (8, 723370), (9, 495272)])
((('10576494', datetime.date(2022, 1, 9)), ['202901', '203902', '900185', '900191', '900242']), [(0, 363430), (1, 245282), (2, 458409), (3, 837481), (4, 995236), (5, 638380), (6, 386587), (7, 322102), (8, 888386), (9, 441294)])
((('11145741', datetime.date(2022, 1, 9)), ['100700871', '103001', '103002', '209103', '900218', '900235', '900256']), [(0, 838412), (1, 1045365), (2, 786256), (3, 686231), (4, 724184), (5, 62333), (6, 178986), (7, 294040), (8, 848525), (9, 885618)])
((('80645664', datetime

In [27]:
#printing cosine_similarity for items
for item in candidate_list:
    if item != 0:
        temp = item[0][1]
        counter = 0
        for t in temp:
            if random_road[dict_word[t]] == 1:
                counter += 1
        print("for car {} similarity is : {}".format(item[0][0][0] , counter/len(temp)))

for car 8826688 similarity is : 0.36363636363636365
for car 10170784 similarity is : 1.0
for car 10576494 similarity is : 0.0
for car 11145741 similarity is : 0.42857142857142855
for car 80645664 similarity is : 0.2
for car 34597954 similarity is : 0.4375
for car 8101667 similarity is : 0.3333333333333333
for car 31403877 similarity is : 0.6
for car 41963248 similarity is : 0.25
for car 64260218 similarity is : 0.55
for car 42017093 similarity is : 0.3
for car 68646988 similarity is : 0.5
for car 96182366 similarity is : 0.3333333333333333
for car 91551931 similarity is : 0.8
for car 73424225 similarity is : 1.0
for car 9905518 similarity is : 0.125
for car 10604039 similarity is : 1.0
for car 32433150 similarity is : 1.0
for car 8543628 similarity is : 0.0
for car 68594944 similarity is : 0.5
for car 13347353 similarity is : 1.0
for car 8586549 similarity is : 0.0
for car 95951571 similarity is : 0.3333333333333333
for car 27507630 similarity is : 1.0
for car 10801491 similarity is : 

In [29]:
#checking for results of part one and LSH part
for item in most_similar:
    for temp in candidate_list:
        if temp != 0:
            temp = temp[0][0][0]
            if item[1] == temp:
                print(item[1])

64260218
